In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()

In [ ]:
using Cropbox
using Test
using Plots
using TimeZones
using DataFramesMeta
using CSV
Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
include("../src/Garlic.jl")

import Dates

In [ ]:
tz = tz"America/Los_Angeles"

Cucumber = @config (
# # CV PHYL ILN GLN LL LER SG SD LTAR LTARa LIR Topt Tceil critPPD
# KM1 134 4 10 65.0 4.70 1.84 122 0 0.4421 0.1003 22.28 34.23 12
    :Phenology => (
        # optimal_temperature = 22.28, # Topt
        optimal_temperature = 25, # Topt
        # ceiling_temperature = 34.23, # Tceil
        ceiling_temperature = 30, # Tceil
        critical_photoperiod = 24, # critPPD
        # critical_photoperiod = 12, # critPPD
        # initial_leaves_at_harvest = 5, # ILN
        # maximum_leaf_initiation_rate = 0.8, # LIR
        maximum_leaf_initiation_rate = 1.16939,
        # maximum_emergence_rate = 0.2,
        maximum_emergence_rate = 0.12,
        # maximum_leaf_initiation_rate = 0.1003, # LIR
        # storage_days = 100, # SD
        # storage_temperature = 5,
        # maximum_phyllochron_asymptote = 0.4421, # LTARa
        maximum_phyllochron_asymptote = 1.45901,
        # leaves_generic = 10, # GLN
        leaves_generic = 50, # GLN
        # storage_days = 122, # SD
        planting_date = ZonedDateTime(2021, 2, 1, tz"Asia/Seoul"), # Y1 sow
        emergence_date = ZonedDateTime(2021, 2, 1, tz"Asia/Seoul"), # Y1 emg
        # scape_removal_date = nothing, # Y2 SR
        detach_to_maintain_max_leaf_number = 19,
        detach_to_maintain_min_leaf_number = 15,
    ),
    :Leaf => (
        maximum_elongation_rate = 4.70, # LER
        minimum_length_of_longest_leaf = 25.0, # LL
        # stay_green = 1.84, # SG
        stay_green = 20,
        length_to_width_ratio = 1,
    ),
    :Carbon => (
# # Rm Yg
# 0.012 0.8
        maintenance_respiration_coefficient = 0.012, # Rm
        synthesis_efficiency = 0.8, # Yg
        partitioning_table = [
        # # root leaf sheath scape bulb
        #   0.00 0.00   0.00  0.00 0.00 ; # seed garlic before germination
        #   0.35 0.30   0.25  0.00 0.10 ; # vegetative stage between germination and scape initiation
        #   0.15 0.15   0.10  0.25 0.35 ; # period between scape initiation and scape appearance
        #   0.05 0.10   0.00  0.35 0.50 ; # period after scape appearance before removal (scape stays intact)
        #   0.05 0.00   0.00  0.00 0.95 ; # period after scape removal (scape appeared and subsequently removed)
        #   0.00 0.00   0.00  0.00 0.00 ; # dead
        # ],
        # root leaf sheath scape bulb
          0.00 0.00   0.00  0.00 0.00 ; # seed garlic before germination
          0.60 0.40   0.00  0.00 0.00 ; # vegetative stage between germination and scape initiation
          0.60 0.40   0.00  0.00 0.00 ; # period between scape initiation and scape appearance
          0.60 0.40   0.00  0.00 0.00 ; # period after scape appearance before removal (scape stays intact)
          0.60 0.40   0.00  0.00 0.00 ; # period after scape removal (scape appeared and subsequently removed)
          0.00 0.00   0.00  0.00 0.00 ; # dead
        ],
    ),
    :Meta => (cultivar = :Cucumber,
        year = 2021,),
    :Weather => (
        # CO2 = 390, # CO2
        CO2 = 500, # CO2
        store = Garlic.loadwea("$(@__DIR__)/../data/Wea2nd.wea", tz"Asia/Seoul"),
    ),
    :Plant => (initial_planting_density = 55,),
    :Calendar => (
        init = ZonedDateTime(2021, 2, 1, tz"Asia/Seoul"), # Y1 bgn
        last = ZonedDateTime(2021, 8, 29, tz"Asia/Seoul"), #Y2 end
    ),
)

P1 = @config :Meta => (; planting_group = 1)
P2 = @config :Meta => (; planting_group = 2)

In [ ]:
obs = CSV.read("$(@__DIR__)/../data/Phenology.csv", DataFrame)

f(s) = s.DAP' in obs.DAP && Dates.hour(s.calendar.time') == 12
#r = simulate(Garlic.Model;
# calibrate(Garlic.Model, obs;
#     config=Cucumber,
#     stop="calendar.count",
#     index=:DAP,
#     target=:leaf_count => :leaves_real,
#     parameters= :Phenology => (;
#         LTARa_max = (0.2, 1.5),
#         maximum_leaf_initiation_rate = (0.2, 1.5),
#     ),
#     snap=f,
#     optim=(:MaxSteps => 10,),
# )

In [ ]:
obs = CSV.read("$(@__DIR__)/../data/Phenology.csv", DataFrame) |> unitfy

r = simulate(Garlic.Model;
    config=Cucumber,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
)
@test r.leaves_initiated[end] > 0

visualize(r, :DAP, [:leaves_appeared, :leaves_mature, :leaves_dropped, :leaves_detached, :leaves_real], kind=:line) |> display # Fig. 3.D
visualize(r, :DAP, :LAI) |> display # Fig. 4.D
visualize(r, :DAP, [:leaf_mass, :total_mass]) |> display
visualize(r, :DAP, [:A_net], kind=:line) |> display
p = visualize(r, :DAP, [:leaves_real], kind=:line)

In [ ]:
manipulate(Garlic.Model, :DAP, [:leaves_appeared, :leaves_mature, :leaves_dropped, :leaves_detached, :leaves_real];
    config = Cucumber,
    parameters = :Phenology => (;
        optimal_temperature = 10:40,
        
    ),
    stop = "calendar.count",
    kind = :line,
)

In [ ]:
evaluate(Garlic.Model, obs; 
    config = Cucumber,
    index = :DAP => :DAP,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
    target = :leaf_count => :leaves_real, metric = :rmse)